# Улучшение прогнозов добавлением обучаемых признаков 

##### Автор: Виктор Китов ([DeepMachineLearning.ru](https://deepmachinelearning.ru))

##### Лицензия: BSD-3-Clause

В этом ноутбуке мы покажем, как добавление новых настраиваемых признаков к исходным повышает качество классификации.

Новые признаки мы будем генерировать, используя следующие модели:
- <u>метод главных компонент</u> — чтобы получить проекции на направления основной вариации данных
- <u>линейный дискриминантный анализ</u> — чтобы получить признаки, специально оптимизированные для разделения классов
- <u>кластеризация K-средних</u> — чтобы кодировать расстояние от объектов до типичным групп в пространстве признаков
- <u>IsolationForest</u>  — чтобы добавить информацию о том, насколько каждый объект нетипичен для датасета

Работать будем с датасетом Digits, состоящим из 1797 изображений рукописных цифр размера 8×8 пикселей. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.base import clone
import warnings
warnings.filterwarnings("ignore")

# КОНСТАНТЫ

In [2]:
K = 2  # Количество компонент для PCA и LDA
CLUSTERS = 10  # Количество кластеров для KMeans

In [3]:
print("Загрузка датасета Digits (цифры 0-9)...")
digits = load_digits()
X = digits.data
y = digits.target

print(f"Обучающая выборка: {X.shape}")
print(f"Классы: {np.unique(y)}")
print()

# Стандартизация данных 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение стандартизованных данных
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Размеры после разделения:")
print(f"  Обучающая: {X_train.shape}")
print(f"  Тестовая: {X_test.shape}")
print()

Загрузка датасета Digits (цифры 0-9)...
Обучающая выборка: (1797, 64)
Классы: [0 1 2 3 4 5 6 7 8 9]

Размеры после разделения:
  Обучающая: (1437, 64)
  Тестовая: (360, 64)



In [4]:
base_classifier = RandomForestClassifier(
    n_estimators=100,
    max_features='sqrt',
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

# МОДЕЛЬ НА ИСХОДНЫХ ПРИЗНАКАХ

In [5]:
model_base = clone(base_classifier)
model_base.fit(X_train, y_train)
y_pred_base = model_base.predict(X_test)
acc_base = accuracy_score(y_test, y_pred_base)

# МОДЕЛЬ ТОЛЬКО НА PCA КОМПОНЕНТАХ

In [6]:
# Создаем PCA компоненты
pca = PCA(n_components=K, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

model_pca_only = clone(base_classifier)
model_pca_only.fit(X_train_pca, y_train)
y_pred_pca_only = model_pca_only.predict(X_test_pca)
acc_pca_only = accuracy_score(y_test, y_pred_pca_only)

# МОДЕЛЬ НА ИСХОДНЫХ + PCA КОМПОНЕНТАХ

In [7]:
# Объединяем исходные признаки с PCA компонентами
X_train_pca_combined = np.hstack([X_train, X_train_pca])
X_test_pca_combined = np.hstack([X_test, X_test_pca])

model_pca_combined = clone(base_classifier)
model_pca_combined.fit(X_train_pca_combined, y_train)
y_pred_pca_combined = model_pca_combined.predict(X_test_pca_combined)
acc_pca_combined = accuracy_score(y_test, y_pred_pca_combined)

# МОДЕЛЬ ТОЛЬКО НА LDA КОМПОНЕНТАХ

In [8]:
# Для LDA определяем максимальное количество компонент
max_lda_components = min(X_train.shape[1], len(np.unique(y_train))-1)
n_lda_components = min(K, max_lda_components)

# Создаем LDA компоненты
lda = LinearDiscriminantAnalysis(n_components=n_lda_components)
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

model_lda_only = clone(base_classifier)
model_lda_only.fit(X_train_lda, y_train)
y_pred_lda_only = model_lda_only.predict(X_test_lda)
acc_lda_only = accuracy_score(y_test, y_pred_lda_only)

# МОДЕЛЬ НА ИСХОДНЫХ + LDA КОМПОНЕНТАХ

In [9]:
print("\n" + "="*60)
print("5. МОДЕЛЬ НА ИСХОДНЫХ ПРИЗНАКАХ + LDA КОМПОНЕНТЫ")
print("="*60)

# Объединяем исходные признаки с LDA компонентами
X_train_lda_combined = np.hstack([X_train, X_train_lda])
X_test_lda_combined = np.hstack([X_test, X_test_lda])

model_lda_combined = clone(base_classifier)
model_lda_combined.fit(X_train_lda_combined, y_train)
y_pred_lda_combined = model_lda_combined.predict(X_test_lda_combined)
acc_lda_combined = accuracy_score(y_test, y_pred_lda_combined)
print(f"Accuracy: {acc_lda_combined:.4f}")
print(f"Количество признаков: {X_train.shape[1]} + {n_lda_components} = {X_train_lda_combined.shape[1]}")


5. МОДЕЛЬ НА ИСХОДНЫХ ПРИЗНАКАХ + LDA КОМПОНЕНТЫ
Accuracy: 0.9778
Количество признаков: 64 + 2 = 66


# МОДЕЛЬ ТОЛЬКО НА РАССТОЯНИЯХ ДО КЛАСТЕРОВ

In [10]:
# Кластеризация KMeans
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=CLUSTERS, random_state=42, n_init=10)
kmeans.fit(X_train)

# Вычисляем расстояния до центроидов кластеров
from sklearn.metrics import pairwise_distances
train_distances = pairwise_distances(X_train, kmeans.cluster_centers_)
test_distances = pairwise_distances(X_test, kmeans.cluster_centers_)

# Используем расстояния как признаки
X_train_kmeans = train_distances  # Размер: (n_samples, CLUSTERS)
X_test_kmeans = test_distances    # Размер: (n_samples, CLUSTERS)

model_kmeans_only = clone(base_classifier)
model_kmeans_only.fit(X_train_kmeans, y_train)
y_pred_kmeans_only = model_kmeans_only.predict(X_test_kmeans)
acc_kmeans_only = accuracy_score(y_test, y_pred_kmeans_only)

# МОДЕЛЬ НА ИСХОДНЫХ ПРИЗНАКАХ + РАССТОЯНИЯХ ДО КЛАСТЕРОВ

In [11]:
# Объединяем исходные признаки с расстояниями до кластеров
X_train_kmeans_combined = np.hstack([X_train, train_distances])
X_test_kmeans_combined = np.hstack([X_test, test_distances])

model_kmeans_combined = clone(base_classifier)
model_kmeans_combined.fit(X_train_kmeans_combined, y_train)
y_pred_kmeans_combined = model_kmeans_combined.predict(X_test_kmeans_combined)
acc_kmeans_combined = accuracy_score(y_test, y_pred_kmeans_combined)

# МОДЕЛЬ НА ИСХОДНЫХ ПРИЗНАКАХ + СТЕПЕНЬ НЕТИПИЧНОСТИ

In [12]:
# Обучаем IsolationForest для получения оценки аномальности
from sklearn.ensemble import IsolationForest
iso_forest = IsolationForest(contamination=0.1, random_state=42, n_jobs=-1)
iso_forest.fit(X_train)

# Получаем оценки аномальности (decision function)
# Чем меньше значение - тем более аномальный объект
train_anomaly_scores = iso_forest.decision_function(X_train)
test_anomaly_scores = iso_forest.decision_function(X_test)

# Масштабируем оценки для лучшей сочетаемости с исходными признаками
from sklearn.preprocessing import StandardScaler
anomaly_scaler = StandardScaler()
train_anomaly_scores_scaled = anomaly_scaler.fit_transform(train_anomaly_scores.reshape(-1, 1))
test_anomaly_scores_scaled = anomaly_scaler.transform(test_anomaly_scores.reshape(-1, 1))

# Объединяем исходные признаки с оценкой аномальности
X_train_anomaly_combined = np.hstack([X_train, train_anomaly_scores_scaled])
X_test_anomaly_combined = np.hstack([X_test, test_anomaly_scores_scaled])

model_anomaly_combined = clone(base_classifier)
model_anomaly_combined.fit(X_train_anomaly_combined, y_train)
y_pred_anomaly_combined = model_anomaly_combined.predict(X_test_anomaly_combined)
acc_anomaly_combined = accuracy_score(y_test, y_pred_anomaly_combined)

# ВЫВОД РЕЗУЛЬТАТОВ

In [13]:
results = [
    ("Исходные признаки", acc_base, X_train.shape[1]),
    (f"Только PCA (# компонент: {K})", acc_pca_only, K),
    (f"Исходные + PCA (# компонент: {K})", acc_pca_combined, X_train_pca_combined.shape[1]),
    (f"Только LDA (# компонент: {n_lda_components})", acc_lda_only, n_lda_components),
    (f"Исходные + LDA (# компонент: {n_lda_components})", acc_lda_combined, X_train_lda_combined.shape[1]),
    (f"Только KMeans (# кластеров: {CLUSTERS})", acc_kmeans_only, CLUSTERS),
    (f"Исходные + KMeans (# кластеров: {CLUSTERS})", acc_kmeans_combined, X_train_kmeans_combined.shape[1]),
    (f"Исходные + аномальность", acc_anomaly_combined, X_train_anomaly_combined.shape[1])
]
summary_df = pd.DataFrame(results, columns=['Метод', 'Accuracy', 'Кол-во признаков'])
summary_df = summary_df.sort_values('Accuracy', ascending=False)

print(summary_df.to_string(index=False))

                              Метод  Accuracy  Кол-во признаков
    Исходные + LDA (# компонент: 2)  0.977778                66
    Исходные + PCA (# компонент: 2)  0.975000                66
            Исходные + аномальность  0.975000                65
Исходные + KMeans (# кластеров: 10)  0.969444                74
                  Исходные признаки  0.961111                64
    Только KMeans (# кластеров: 10)  0.861111                10
        Только LDA (# компонент: 2)  0.702778                 2
        Только PCA (# компонент: 2)  0.511111                 2
